# Laborator 4 - Data Normalization and SVM

## Helper methods & examples

In [2]:
from sklearn import preprocessing
import numpy as np

x_train = np.array([[1, -1, 2], [2, 0, 0], [0, 1, -1]], dtype=np.float64)
x_test = np.array([[-1, 1, 0]], dtype=np.float64)

# facem statisticile pe datele de antrenare
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
# afisam media
print(scaler.mean_) # => [1. 0. 0.33333333]
# afisam deviatia standard
print(scaler.scale_) # => [0.81649658 0.81649658 1.24721913]

# scalam datele de antrenare
scaled_x_train = scaler.transform(x_train)
print(scaled_x_train) # => [[0. -1.22474487 1.33630621]
# [1.22474487 0. -0.26726124]
# [-1.22474487 1.22474487 -1.06904497]]

# scalam datele de test
scaled_x_test = scaler.transform(x_test)
print(scaled_x_test) # => [[-2.44948974 1.22474487 -0.26726124]]

[1.         0.         0.33333333]
[0.81649658 0.81649658 1.24721913]
[[ 0.         -1.22474487  1.33630621]
 [ 1.22474487  0.         -0.26726124]
 [-1.22474487  1.22474487 -1.06904497]]
[[-2.44948974  1.22474487 -0.26726124]]


## Exercitii

1. Descarcati si cititi datele de train si de test (atentie la formatul mesajelor)

In [3]:
import numpy as np

training_sentences = np.load("./data/training_sentences.npy", allow_pickle=True)
test_sentences = np.load("./data/test_sentences.npy", allow_pickle=True)
train_labels = np.load("./data/training_labels.npy", allow_pickle=True)
test_labels = np.load("./data/test_labels.npy", allow_pickle=True)

2. Definiți funcția normalize_data(train_data, test_data, type=None) care primește ca
parametri datele de antrenare, respectiv de testare și tipul de normalizare ({None,
‘standard’, ‘l1’, ‘l2’}) și întoarce aceste date normalizate.


In [4]:
from sklearn.preprocessing import StandardScaler, Normalizer
import numpy as np

def normalize_data(train_data, test_data, type=None):
    if type is None:
        return (train_data, test_data)
    
    scaler = StandardScaler()
    scaler.fit(train_data)
    
    standard_train = scaler.transform(train_data)
    standard_test = scaler.transform(test_data)
    
    if type == "l1" or type == "l2":
        train_array = np.array(standard_train)
        test_array = np.array(standard_test)
        
        transformer = Normalizer(type)
        normalized_train = transformer.transform(train_array)
        normalized_test = transformer.transform(test_array)
        
        return (normalized_train, normalized_test)
    
    return (standard_train, standard_test)
    

3. Definiți clasa BagOfWords în al cărui constructor se inițializează vocabularul (un
dicționar gol). În cadrul ei implementați metoda build_vocabulary(self, data) care
primește ca parametru o listă de mesaje(listă de liste de strings) și construiește
vocabularul pe baza acesteia. Cheile dicționarului sunt reprezentate de cuvintele din
eseuri, iar valorile de id-urile unice atribuite cuvintelor. Pe lângă vocabularul pe care-l
construiți, rețineți și o listă cu cuvintele în ordinea adăugării în vocabular.
Afișați dimensiunea vocabularul construit (9522).
OBS. Vocabularul va fi construit doar pe baza datelor din setul de antrenare.


4. Definiți metoda get_features(self, data) care primește ca parametru o listă de
mesaje de dimensiune num_samples(listă de liste de strings) și returnează o matrice
de dimensiune (num_samples x dictionary_length) definită astfel:
features(sample_idx, word_idx) = numarul de aparitii al
cuvantului cu id − ul word_idx in documentul sample_idx

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

class BagOfWords:
    def __init__(self):
        self.vocabulary = {}             # Dictionar {word: id}
        self.word_list = []              # Lista in ordinea adaugarii

    def build_vocabulary(self, data):
        unique_id = 1
        for message in data:
            for word in message:
                if self.vocabulary.get(word, None) is None:
                    self.vocabulary[word] = unique_id
                    unique_id += 1
                    self.word_list.append(word)
        print(len(self.vocabulary))
    
    def get_features(self, data):
        features = np.zeros((len(data), len(self.word_list)))
        for i, message in enumerate(data):
            for j, word in enumerate(self.word_list):
                features[i][j] = message.count(word)
        return features

5. Cu ajutorul funcțiilor definite anterior, obțineți reprezentările BOW pentru mulțimea de
antrenare și testare, apoi normalizați-le folosind norma “L2”.


In [6]:
BOW = BagOfWords()
BOW.build_vocabulary(training_sentences)

train_features = BOW.get_features(training_sentences)
test_features = BOW.get_features(test_sentences)
train_data, test_data = normalize_data(train_features, test_features, type = "l2")

9522


6. a) Antrenați un SVM cu kernel linear care să clasifice mesaje în mesaje
spam/non-spam. Pentru parametrul C setați valoarea 1.

In [7]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC()
svc_model.fit(train_features, train_labels)

estimated_labels = svc_model.predict(test_features)

print(estimated_labels[1:100])

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]


b) Calculați acuratețea și
F1-score pentru mulțimea de testare.


In [8]:
print(svc_model.score(test_features, test_labels))

TP, FP, FN = 0, 0, 0

for i, label in enumerate(estimated_labels):
    TP += label == 1 and test_labels[i] == 1
    FP += label == 1 and test_labels[i] == 0
    FN += label == 0 and test_labels[i] == 1

print(f"F1-score: {(2 * TP) / (2 * TP + FP + FN)}")

0.9853260869565217
F1-score: 0.9443298969072165


c) Afișați cele mai negative (spam) 10 cuvinte și cele mai pozitive (non-spam) 10
cuvinte.
the first 10 negative words are ['Text' 'To' 'mobile' 'CALL' 'FREE' 'txt' '&' 'Call' 'Txt'
'STOP']
the first 10 positive words are ['&lt#&gt' 'me' 'i' 'Going' 'him' 'Ok' 'I' 'Ill' 'my' 'Im']

In [9]:
coefficients = svc_model.coef_[0]
sorted_idx = coefficients.argsort()
print([BOW.word_list[idx] for idx in sorted_idx[0:10]])
print([BOW.word_list[idx] for idx in sorted_idx[-10:]][::-1])

['&lt#&gt', 'him', 'Oh', 'Alright', 'me', 'always', 'right', 'It', 'Ill', 'Waiting']
['ringtoneking', '84484', 'REAL', 'won', 'FREE>RingtoneReply', '85233', 'Txt', 'httptms', 'widelivecomindex', 'For']
